In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
llm = ChatOpenAI(
    temperature=0.1,
) 

cache_dir = LocalFileStore("./.cache/")



splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=500,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader('./files/00713_small.txt', encoding='utf-8')

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,cache_dir
)


vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()   

prompt= ChatPromptTemplate.from_messages([
    (
        "system", 
        "당신은 훌륭한 조력자 입니다. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
    ),
    ("human", "{question}"),
]
)

chain = (
    {
        "context":retriver,
        "question":RunnablePassthrough(),
        "extra":RunnablePassthrough()
    } 
    | prompt 
    | llm
)




#chain.run("맹견의 범위는 어떻게 되나요?")
#chain.run("반려동물의 범위 어떻게 되나요?")
chain.invoke("동물복지위원회 위원 자격요건이 어떻게 되나요?")

Created a chunk of size 807, which is longer than the specified 500
Created a chunk of size 1116, which is longer than the specified 500


AIMessage(content='동물복지위원회 위원 자격요건은 다음과 같습니다:\n1. 동물실험윤리위원회의 위원\n2. 동물보호ㆍ동물복지에 관한 학식과 경험이 풍부한 영업자\n3. 명예동물보호관으로 위촉된 사람\n4. 축산단체의 대표로서 동물보호ㆍ동물복지에 관한 학식과 경험이 풍부한 사람\n5. 변호사\n6. 법학 또는 동물보호ㆍ동물복지를 담당하는 학교의 조교수 이상\n7. 농림축산식품부장관이 동물보호ㆍ동물복지에 관한 학식과 경험이 풍부하다고 인정하는 사람')